# Building the Transformer from Scratch

In this notebook, we'll be implementing the famous Transformer architecture from scratch.

The code is based off of the following repos/blog posts:

- [attention-is-all-you-need-pytorch](https://github.com/jadore801120/attention-is-all-you-need-pytorch)
- [pytorch-pretrained-BERT](https://github.com/huggingface/pytorch-pretrained-BERT)
- [The Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention.html) 

Thanks so much to their authors!

In [1]:
import torch
import torch.nn as nn
import numpy as np

One of the keys to understanding how any model works is understanding how the shapes of the tensors change during the processing of each part. We'll be using the logging module to output debugging information to help our understanding.

In [2]:
import logging
logger = logging.getLogger("tensor_shapes")
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
# if you want the model to continuously print tensor shapes, set to DEBUG!
logger.setLevel(1)

In [3]:
import inspect
def getclass():
    stack = inspect.stack()
    return stack[3][0].f_locals["self"].__class__

# A helper function to check how tensor sizes change
def log_size(tsr: torch.Tensor, name: str):
    cls = getclass()
    logger.log(level=cls.level, msg=f"[{cls.__name__}] {name} size={tsr.shape}")

We'll use logging levels to control the modules we receive output from. The lower the logging level, the more tensor information you'll get. Feel free to play around!

In [4]:
from enum import IntEnum
# Control how much debugging output we want
class TensorLoggingLevels(IntEnum):
    attention = 1
    attention_head = 2
    multihead_attention_block = 3
    enc_dec_block = 4
    enc_dec = 5

We'll be using an enum to refer to dimensions whenever possible to improve readability.

In [5]:
class Dim(IntEnum):
    batch = 0
    seq = 1
    feature = 2

# Components

### Scaled dot product attention

The Transformer is an attention-based architecture. The attention used in the Transformer is the scaled dot product attention, represented by the following formula.

$$ \textrm{Attention}(Q, K, V) = \textrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V $$

![image](https://i2.wp.com/mlexplained.com/wp-content/uploads/2017/12/scaled_dot_product_attention.png?zoom=2&w=750)

In [6]:
import math

class ScaledDotProductAttention(nn.Module):
    level = TensorLoggingLevels.attention # Logging level: 
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        d_k = k.size(-1) # get the size of the key
        assert q.size(-1) == d_k

        # compute the dot product between queries and keys for
        # each batch and position in the sequence
        attn = torch.bmm(q, k.transpose(Dim.seq, Dim.feature)) # (Batch, Seq, Seq)
        # we get an attention score between each position in the sequence
        # for each batch

        # scale the dot products by the dimensionality (see the paper for why we do this!)
        attn = attn / math.sqrt(d_k)
        # normalize the weights across the sequence dimension
        # (Note that since we transposed, the sequence and feature dimensions are switched)
        attn = torch.exp(attn)
        log_size(attn, "attention weight") # (Batch, Seq, Seq)
        
        # fill attention weights with 0s where padded
        if mask is not None: attn = attn.masked_fill(mask, 0)
        attn = attn / attn.sum(dim=-1, keepdim=True)
        attn = self.dropout(attn)
        output = torch.bmm(attn, v) # (Batch, Seq, Feature)
        log_size(output, "attention output size") # (Batch, Seq, Seq)
        return output

In [7]:
attn = ScaledDotProductAttention()

In [8]:
q = torch.rand(5, 10, 20)
k = torch.rand(5, 10, 20)
v = torch.rand(5, 10, 20)

In [9]:
attn(q, k, v)

[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])


tensor([[[0.4155, 0.5476, 0.5082, 0.2934, 0.5051, 0.4386, 0.3601, 0.5725,
          0.6110, 0.4794, 0.5212, 0.5381, 0.3694, 0.4678, 0.5721, 0.4100,
          0.5772, 0.6029, 0.4823, 0.5055],
         [0.4003, 0.4914, 0.4948, 0.3152, 0.4737, 0.5248, 0.4424, 0.5263,
          0.5571, 0.4967, 0.5839, 0.4387, 0.4047, 0.4821, 0.5940, 0.3094,
          0.4735, 0.5127, 0.4548, 0.4922],
         [0.4476, 0.4908, 0.5069, 0.3254, 0.4825, 0.5062, 0.4613, 0.6021,
          0.5678, 0.5532, 0.5904, 0.4289, 0.4781, 0.4523, 0.6046, 0.4032,
          0.5579, 0.6051, 0.4195, 0.5306],
         [0.1954, 0.3446, 0.2871, 0.1605, 0.3742, 0.2639, 0.2238, 0.3539,
          0.4962, 0.3058, 0.3466, 0.3992, 0.2556, 0.3081, 0.4143, 0.3448,
          0.4521, 0.4266, 0.2886, 0.2991],
         [0.4436, 0.5463, 0.5449, 0.3344, 0.5127, 0.5633, 0.4302, 0.5472,
          0.6020, 0.5208, 0.6171, 0.4898, 0.4234, 0.5031, 0.6462, 0.3333,
          0.4936, 0.5137, 0.5001, 0.4946],
         [0.3570, 0.4294, 0.3758, 0.2352, 0.4

### Multi-Head Attention

Now, we turn to the core component in the Transformer architecture: the multi-head attention block. This block applies linear transformations to the input, then applies scaled dot product attention.

![image](https://i2.wp.com/mlexplained.com/wp-content/uploads/2017/12/multi_head_attention.png?zoom=2&resize=224%2C293)

In [10]:
class AttentionHead(nn.Module):
    level = TensorLoggingLevels.attention_head
    def __init__(self, d_model, d_feature, dropout=0.1):
        super().__init__()
        # We will assume the queries, keys, and values all have the same feature size
        self.attn = ScaledDotProductAttention(dropout)
        self.query_tfm = nn.Linear(d_model, d_feature)
        self.key_tfm = nn.Linear(d_model, d_feature)
        self.value_tfm = nn.Linear(d_model, d_feature)

    def forward(self, queries, keys, values, mask=None):
        Q = self.query_tfm(queries) # (Batch, Seq, Feature)
        K = self.key_tfm(keys) # (Batch, Seq, Feature)
        V = self.value_tfm(values) # (Batch, Seq, Feature)
        log_size(Q, "queries, keys, vals")
        # compute multiple attention weighted sums
        x = self.attn(Q, K, V)
        return x

In [11]:
attn_head = AttentionHead(20, 20)
attn_head(q, k, v)

[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[ScaledDotProductAttention] attention weight size=torch.Size([5, 10, 10])
[ScaledDotProductAttention] attention output size size=torch.Size([5, 10, 20])


tensor([[[-2.6996e-01,  1.8375e-04,  9.1657e-01, -2.1066e-01, -3.7544e-03,
          -4.2488e-01, -4.4396e-02, -4.1814e-01, -1.4343e-01,  6.8965e-01,
          -2.8909e-01,  2.2854e-01,  6.5507e-01,  5.3401e-01,  2.6055e-01,
           4.5195e-02, -3.3342e-01, -2.5821e-01,  4.8853e-01, -7.6121e-01],
         [-2.6549e-01,  4.9494e-03,  9.1689e-01, -2.1464e-01, -2.4507e-03,
          -4.2283e-01, -4.4367e-02, -4.1772e-01, -1.3946e-01,  6.8473e-01,
          -2.8738e-01,  2.2760e-01,  6.5208e-01,  5.3310e-01,  2.5720e-01,
           4.1898e-02, -3.3669e-01, -2.5638e-01,  4.8931e-01, -7.6089e-01],
         [-2.5423e-01,  1.6306e-02,  8.4685e-01, -2.0307e-01,  2.9387e-02,
          -4.0364e-01, -3.1224e-02, -3.8628e-01, -1.5168e-01,  6.4818e-01,
          -2.7544e-01,  2.1792e-01,  5.7187e-01,  5.1569e-01,  2.7195e-01,
           5.3186e-02, -2.8877e-01, -2.3342e-01,  4.4399e-01, -7.0993e-01],
         [-2.6786e-01,  4.8589e-03,  9.1855e-01, -2.1370e-01, -4.9534e-04,
          -4.2614e-01,

The multi-head attention block simply applies multiple attention heads, then concatenates the outputs and applies a single linear projection.

In [12]:
# We'll supress logging from the scaled dot product attention now
logger.setLevel(TensorLoggingLevels.attention_head)

In [13]:
class MultiHeadAttention(nn.Module):
    level = TensorLoggingLevels.multihead_attention_block
    def __init__(self, d_model, d_feature, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.d_feature = d_feature
        self.n_heads = n_heads
        # in practice, d_model == d_feature * n_heads
        assert d_model == d_feature * n_heads

        # Note that this is very inefficient:
        # I am merely implementing the heads separately because it is 
        # easier to understand this way
        self.attn_heads = nn.ModuleList([
            AttentionHead(d_model, d_feature, dropout) for _ in range(n_heads)
        ])
        self.projection = nn.Linear(d_feature * n_heads, d_model) 
    
    def forward(self, queries, keys, values, mask=None):
        log_size(queries, "Input queries")
        x = [attn(queries, keys, values, mask=mask) # (Batch, Seq, Feature)
             for i, attn in enumerate(self.attn_heads)]
        log_size(x[0], "output of single head")
        
        # reconcatenate
        x = torch.cat(x, dim=Dim.feature) # (Batch, Seq, D_Feature * n_heads)
        log_size(x, "concatenated output")
        x = self.projection(x) # (Batch, Seq, D_Model)
        log_size(x, "projected output")
        return x

In [14]:
heads = MultiHeadAttention(20 * 8, 20, 8)
heads(q.repeat(1, 1, 8), 
      k.repeat(1, 1, 8), 
      v.repeat(1, 1, 8))

[MultiHeadAttention] Input queries size=torch.Size([5, 10, 160])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[AttentionHead] queries, keys, vals size=torch.Size([5, 10, 20])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 20])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 160])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 160])


tensor([[[ 0.3042,  0.1619,  0.0106,  ..., -0.1427,  0.0059,  0.1014],
         [ 0.2979,  0.1736,  0.0117,  ..., -0.1062,  0.0062,  0.1129],
         [ 0.2891,  0.1671, -0.0147,  ..., -0.0910,  0.0428,  0.1291],
         ...,
         [ 0.2688,  0.1477, -0.0178,  ..., -0.0798,  0.0340,  0.1409],
         [ 0.2719,  0.2131, -0.0258,  ..., -0.1069,  0.0571,  0.1530],
         [ 0.3113,  0.1642, -0.0201,  ..., -0.1078,  0.0338,  0.1403]],

        [[ 0.3256,  0.0920, -0.0382,  ..., -0.0416, -0.0111,  0.1374],
         [ 0.3092,  0.1090, -0.0048,  ..., -0.0738, -0.0060,  0.1081],
         [ 0.3076,  0.1170, -0.0270,  ..., -0.0978,  0.0117,  0.1132],
         ...,
         [ 0.3159,  0.1064, -0.0193,  ..., -0.0508, -0.0045,  0.0988],
         [ 0.3430,  0.0834, -0.0113,  ..., -0.0589, -0.0063,  0.1070],
         [ 0.3304,  0.0820, -0.0062,  ..., -0.0756, -0.0149,  0.0951]],

        [[ 0.2724,  0.1273, -0.0339,  ..., -0.0428,  0.0219,  0.1351],
         [ 0.3204,  0.1184,  0.0022,  ..., -0

### The Encoder

With these core components in place, implementing the encoder is pretty easy.

![image](https://i2.wp.com/mlexplained.com/wp-content/uploads/2017/12/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2017-12-29-19.14.41.png?w=273)

The encoder consists of the following components:
- A multi-head attention block
- A simple feedforward neural network

These components are connected using residual connections and layer normalization

In [15]:
# We'll supress logging from the individual attention heads
logger.setLevel(TensorLoggingLevels.multihead_attention_block)

Layer normalization is similar to batch normalization, but normalizes across the feature dimension instead of the batch dimension.

![image](https://i1.wp.com/mlexplained.com/wp-content/uploads/2018/01/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2018-01-11-11.48.12.png?w=1500)

In [16]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-8):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

The encoder just stacks these together

In [17]:
class EncoderBlock(nn.Module):
    level = TensorLoggingLevels.enc_dec_block
    def __init__(self, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        self.attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.layer_norm1 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.position_wise_feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )
        self.layer_norm2 = LayerNorm(d_model)
        
    def forward(self, x, mask=None):
        log_size(x, "Encoder block input")
        att = self.attn_head(x, x, x, mask=mask)
        log_size(x, "Attention output")
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm1(att))
        # Apply position-wise feedforward network
        pos = self.position_wise_feed_forward(x)
        log_size(x, "Feedforward output")
        # Apply normalization and residual connection
        x = x + self.dropout(self.layer_norm2(pos))
        log_size(x, "Encoder size output")
        return x

In [18]:
enc = EncoderBlock()

In [19]:
enc(torch.rand(5, 10, 512))

[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[EncoderBlock] Attention output size=torch.Size([5, 10, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 10, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 10, 512])


tensor([[[ 5.5778e-01,  1.2996e-01, -2.0364e+00,  ...,  9.2855e-01,
           3.0538e-01, -1.9435e+00],
         [ 1.7227e+00, -1.8700e-01, -1.9670e+00,  ...,  1.5087e+00,
           1.0436e+00,  4.1258e-02],
         [ 1.2205e+00, -2.2935e-02, -2.5246e+00,  ...,  2.7623e+00,
           2.8276e-02, -3.0901e+00],
         ...,
         [ 7.7646e-01, -4.2103e-01, -1.8793e-01,  ...,  1.0549e+00,
           9.1001e-01,  2.9219e-02],
         [ 6.2856e-01,  6.0925e-01, -2.1385e+00,  ...,  2.0138e+00,
           6.0836e-01, -2.4650e+00],
         [ 1.3225e+00, -6.8761e-01, -1.4338e+00,  ...,  2.3427e+00,
           9.0309e-01, -1.8249e+00]],

        [[ 1.9393e+00, -3.6200e-03, -2.3605e+00,  ...,  1.7602e+00,
           8.6034e-01,  3.1441e-01],
         [ 1.1189e+00,  6.1461e-02, -1.6022e+00,  ...,  2.7324e+00,
           7.8264e-01, -3.3045e+00],
         [ 2.1796e-01,  3.5294e-02, -2.7486e+00,  ...,  3.2831e-01,
          -4.0683e-02, -2.8061e+00],
         ...,
         [ 1.4822e+00, -8

The encoder consists of 6 consecutive encoder blocks, so can simply be implemented like the following

In [20]:
class TransformerEncoder(nn.Module):
    level = TensorLoggingLevels.enc_dec
    def __init__(self, n_blocks=6, d_model=512,
                 n_heads=8, d_ff=2048, dropout=0.1):
        super().__init__()
        self.encoders = nn.ModuleList([
            EncoderBlock(d_model=d_model, d_feature=d_model // n_heads,
                         d_ff=d_ff, dropout=dropout)
            for _ in range(n_blocks)
        ])
    
    def forward(self, x: torch.FloatTensor, mask=None):
        for encoder in self.encoders:
            x = encoder(x)
        return x

### The Decoder

The decoder is mostly the same as the encoder. There's just one additional multi-head attention block that takes the target sentence as input.

![image](https://i1.wp.com/mlexplained.com/wp-content/uploads/2017/12/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2017-12-29-19.14.47.png?w=287)

The keys and values are the outputs of the encoder, and the queries are the outputs of the multi-head attention over the target entence embeddings.

In [21]:
class DecoderBlock(nn.Module):
    level = TensorLoggingLevels.enc_dec_block
    def __init__(self, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        self.masked_attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.attn_head = MultiHeadAttention(d_model, d_feature, n_heads, dropout)
        self.position_wise_feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )

        self.layer_norm1 = LayerNorm(d_model)
        self.layer_norm2 = LayerNorm(d_model)
        self.layer_norm3 = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_out, 
                src_mask=None, tgt_mask=None):
        # Apply attention to inputs
        att = self.masked_attn_head(x, x, x, mask=src_mask)
        x = x + self.dropout(self.layer_norm1(att))
        # Apply attention to the encoder outputs and outputs of the previous layer
        att = self.attn_head(queries=x, keys=enc_out, values=enc_out, mask=tgt_mask)
        x = x + self.dropout(self.layer_norm2(att))
        # Apply position-wise feedforward network
        pos = self.position_wise_feed_forward(x)
        x = x + self.dropout(self.layer_norm2(pos))
        return x

In [22]:
dec = DecoderBlock()
dec(torch.rand(5, 10, 512), enc(torch.rand(5, 10, 512)))

[EncoderBlock] Encoder block input size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[EncoderBlock] Attention output size=torch.Size([5, 10, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 10, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 10, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 10, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 10, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 10, 64])
[MultiHeadAttention] concatenated output size=torch.Siz

tensor([[[ 1.6580,  1.6086,  2.2659,  ..., -1.1474,  1.6117, -0.2441],
         [ 2.4482, -0.9475,  1.9422,  ..., -1.0947,  0.6952,  0.7469],
         [ 1.5224,  0.5489,  1.5800,  ...,  0.1848,  1.8750,  0.6865],
         ...,
         [ 1.5526, -0.5386,  0.2448,  ..., -1.5395,  0.8934, -0.2828],
         [ 2.1378,  0.9989,  1.6901,  ...,  0.7147,  1.6487, -0.5346],
         [ 1.7555,  0.1182,  1.9243,  ..., -0.6435,  1.5094,  0.7406]],

        [[ 2.3763,  0.2463,  1.3593,  ..., -0.6498,  1.9941,  0.2738],
         [ 2.2653,  0.3445,  1.4671,  ..., -1.0508,  1.6819,  0.6277],
         [ 1.2639, -0.7048,  1.5885,  ...,  0.7334,  2.5985, -0.2863],
         ...,
         [ 1.9279, -0.3174,  2.3046,  ..., -1.0829,  1.9061,  1.3080],
         [ 1.3932, -0.2881,  1.0695,  ...,  0.2407,  2.4994,  0.3057],
         [ 1.7608,  0.2705,  1.4249,  ...,  0.1931,  2.0356,  0.5568]],

        [[ 1.6962, -0.1480,  1.3472,  ..., -0.8366,  1.8088,  0.7894],
         [ 1.3533, -0.8355,  1.2008,  ..., -0

Again, the decoder is just a stack of the underlying block so is simple to implement.

In [23]:
class TransformerDecoder(nn.Module):
    level = TensorLoggingLevels.enc_dec
    def __init__(self, n_blocks=6, d_model=512, d_feature=64,
                 d_ff=2048, n_heads=8, dropout=0.1):
        super().__init__()
        self.position_embedding = PositionalEmbedding(d_model)
        self.decoders = nn.ModuleList([
            DecoderBlock(d_model=d_model, d_feature=d_model // n_heads,
                         d_ff=d_ff, dropout=dropout)
            for _ in range(n_blocks)
        ])
        
    def forward(self, x: torch.FloatTensor, 
                enc_out: torch.FloatTensor, 
                src_mask=None, tgt_mask=None):
        for decoder in self.decoders:
            x = decoder(x, enc_out, src_mask=src_mask, tgt_mask=tgt_mask)
        return x

### Positional Embeddings

Attention blocks are just simple matrix multiplications: therefore they don't have any notion of order! The Transformer explicitly adds positional information via the positional embeddings.

In [24]:
class PositionalEmbedding(nn.Module):
    level = 1
    def __init__(self, d_model, max_len=512):
        super().__init__()        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.weight = nn.Parameter(pe, requires_grad=False)
        
    def forward(self, x):
        return self.weight[:, :x.size(1), :] # (1, Seq, Feature)

In [25]:
class WordPositionEmbedding(nn.Module):
    level = 1
    def __init__(self, vocab_size, d_model=512):
        super().__init__()
        self.word_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = PositionalEmbedding(d_model)
        
    def forward(self, x: torch.LongTensor, mask=None) -> torch.FloatTensor:
        return self.word_embedding(x) + self.position_embedding(x)

In [26]:
emb = WordPositionEmbedding(1000)
encoder = TransformerEncoder()

In [27]:
encoder(emb(torch.randint(1000, (5, 30))))

[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 30, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 30, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 30, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[MultiHeadAttention] Input queries size=torch.Size([5, 30, 512])
[MultiHeadAttention] output of single head size=torch.Size([5, 30, 64])
[MultiHeadAttention] concatenated output size=torch.Size([5, 30, 512])
[MultiHeadAttention] projected output size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[

tensor([[[-3.9217e-01,  1.2299e+00,  3.3396e+00,  ..., -4.4087e-01,
           5.5446e+00,  1.9942e+00],
         [ 2.6970e+00, -1.0296e+00,  8.6089e+00,  ...,  2.3508e+00,
           3.8681e+00,  1.0141e+00],
         [-8.7528e-01,  5.3640e-01,  3.5934e+00,  ...,  1.9303e+00,
           4.5249e+00,  6.1926e-01],
         ...,
         [-1.2338e+00, -7.6097e-01,  6.1613e+00,  ..., -1.5663e+00,
           3.9485e+00,  2.4136e+00],
         [-1.5244e-01,  8.5849e-01,  4.2663e+00,  ..., -1.2305e+00,
           4.4364e+00, -6.5651e-01],
         [-1.3939e+00,  2.2442e+00,  3.2477e+00,  ...,  2.9532e+00,
           2.6887e+00, -1.9719e-01]],

        [[ 1.0720e+00,  5.2130e+00, -1.9961e+00,  ..., -2.8311e+00,
           3.1156e+00,  3.6057e+00],
         [ 8.9173e-01,  1.8281e+00,  4.4644e+00,  ..., -2.8587e+00,
           1.9145e+00,  8.4519e-01],
         [ 2.6354e+00,  2.0722e+00, -9.4354e-01,  ..., -4.2711e+00,
           2.7082e-01,  3.2981e+00],
         ...,
         [ 2.3612e+00, -3

### Putting it All Together

Let's put everything together now.

![image](https://camo.githubusercontent.com/88e8f36ce61dedfd2491885b8df2f68c4d1f92f5/687474703a2f2f696d6775722e636f6d2f316b72463252362e706e67)

In [28]:
# We'll supress logging from the scaled dot product attention now
logger.setLevel(TensorLoggingLevels.enc_dec_block)

In [29]:
emb = WordPositionEmbedding(1000)
encoder = TransformerEncoder()
decoder = TransformerDecoder()

In [30]:
src_ids = torch.randint(1000, (5, 30))
tgt_ids = torch.randint(1000, (5, 30))
x = encoder(emb(src_ids))
decoder(emb(tgt_ids), x)

[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder block input size=torch.Size([5, 30, 512])
[EncoderBlock] Attention output size=torch.Size([5, 30, 512])
[EncoderBlock] Feedforward output size=torch.Size([5, 30, 512])
[EncoderBlock] Encoder size output size=t

tensor([[[-0.9897, -0.6359, -4.6231,  ...,  6.9500, -2.5999, -1.1461],
         [ 4.7274,  2.4179, -1.3643,  ...,  3.7251, -5.0835, -1.2877],
         [ 1.3116, -2.8317, -0.4552,  ...,  6.3076, -2.6063, -1.2082],
         ...,
         [-1.6635,  1.0923, -3.2574,  ...,  5.4824,  7.1426, -1.6863],
         [-5.4686, -5.1602,  2.1902,  ...,  4.9131, -0.5623, -1.3769],
         [-2.0100, -0.0638, -0.5601,  ...,  8.5894, -5.7065, -1.6143]],

        [[ 3.1028,  2.0424,  3.1762,  ...,  4.8382,  0.1979, -1.9995],
         [ 6.9066,  2.4632, -0.3387,  ...,  7.5088,  3.6147, -1.4422],
         [ 0.4897, -1.3431,  1.0123,  ...,  7.9888, -1.5388,  0.0488],
         ...,
         [ 4.1120, -0.1403,  4.1639,  ...,  7.6805, -2.7701, -3.9894],
         [ 4.0688, -0.4527,  6.4777,  ...,  8.5668,  1.8083, -2.8293],
         [ 1.0619, -0.8440,  4.5008,  ...,  7.7869, -0.6064, -5.2547]],

        [[ 3.5427, -2.0393, -5.0770,  ...,  8.2788, -0.4944, -0.8862],
         [ 4.2967,  0.6085, -2.1398,  ...,  6